In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import math
import cvxpy as cp

from decentr_quad_no_constraint import *
import ciropt as co

In [2]:
# random seed
np.random.seed(108)

# data generation
problem_spec = {}
problem_spec['n_node'] = 6
problem_spec['vector_size'] = 10
print(problem_spec)
problem_data = data_generation(problem_spec)
network_data = graph_generation_nodes6()

problem_data['rho'] = 0.01

problem_spec['sc_eps'] = 1

# PG-EXTRA hyperparameters
problem_data['alpha'] = 2.5
problem_data['itr_num'] = 1000

{'n_node': 6, 'vector_size': 10}


In [3]:
# print(problem_data['Q'][1])
# print(problem_data['b'][1])

x = cp.Variable(problem_spec['vector_size'])
# f = problem_data['b'][1] @ x

Q = problem_data['Q']
b = problem_data['b']

# print(x @ Q[1] @ x.T)
# print(b[1] @ x.T)
for j in range(problem_spec['n_node']):
    U, s, V = np.linalg.svd(Q[j])
    print([np.max(s), np.min(s)])

[26.26567379766173, 0.013573752867480244]
[29.280202416421186, 0.002181332707638855]
[24.174188713545213, 0.014584563320790805]
[26.90974964065195, 0.0013239009890112412]
[22.017869951539062, 0.002533092625330057]
[24.979905919445166, 0.01438364895100382]


In [ ]:
nx.draw(network_data["G"], node_color='skyblue', with_labels=True)
# plt.savefig("plots/graph_n6.pdf")

In [ ]:
f_star, x_opt_star = cvx_quad_no_constraint(problem_spec, problem_data)
print(f"{f_star=}, {np.linalg.norm(x_opt_star)=}")

In [ ]:
Q = problem_data['Q']
b = problem_data['b']
 
vector_size = problem_spec['vector_size']
n_node = problem_spec['n_node']
eps = problem_spec['sc_eps']
prox_operators = []
fi_operators = []

# Define functions and proximal operators to implement the method
for jj in range(n_node):
    # bj = b[jj*vector_size : (jj+1)*vector_size]
    prox_operators += [lambda z, alpha:  cvx_prox_fj_quad_no_constraint(z, alpha, problem_spec, problem_data, jj, eps=1e-4)]
    fi_operators += [lambda x_kj : (1/2 * x_kj @ Q[jj] @ x_kj.T + b[jj] @ x_kj)[0] ]

    # z = np.random.randn(vector_size)
    # x1 = prox_fj_geom_mean(z, 1, bj)
    # x2 = cvx_prox_fj_quad_no_constraint(z, 1, bj, vector_size)
    # assert np.linalg.norm(x1 - x2) / np.linalg.norm(x1) < 1e-5

In [ ]:
# params={"R":18, "Inductance":10, "Capacitance":1, "h":1.0541046971920436}
# dadmm_c_err_opt_star, dadmm_c_err_opt_reldiff, dadmm_c_const_vio, dadmm_c_f_reldiff = co.dadmm("cir_dadmm_c", problem_spec, problem_data, 
#                                                                                 network_data, x_opt_star, f_star, prox_operators=prox_operators,
#                                                                                 fi_operators=fi_operators,
#                                                                                 params=params, printing=True, freq=200)

In [ ]:
params={"R":2, "Inductance":4, "Capacitance":4, "h":2.764408907848405}
dadmm_c_err_opt_star, dadmm_c_err_opt_reldiff, dadmm_c_const_vio, dadmm_c_f_reldiff = co.dadmm("cir_dadmm_c", problem_spec, problem_data, 
                                                                                network_data, x_opt_star, f_star, prox_operators=prox_operators,
                                                                                fi_operators=fi_operators,
                                                                                params=params, printing=True, freq=200)

In [ ]:
# R = 10
# admm_err_opt_star, admm_err_opt_reldiff, admm_const_vio, admm_f_reldiff = co.dadmm("dadmm", problem_spec, problem_data, 
#                                                                                 network_data, x_opt_star, f_star, prox_operators=prox_operators,
#                                                                                 fi_operators=fi_operators,
#                                                                                 params={"R":R}, printing=True, freq=200)

In [ ]:
R = 2
admm_err_opt_star, admm_err_opt_reldiff, admm_const_vio, admm_f_reldiff = co.dadmm("dadmm", problem_spec, problem_data, 
                                                                                network_data, x_opt_star, f_star, prox_operators=prox_operators,
                                                                                fi_operators=fi_operators,
                                                                                params={"R":R}, printing=True, freq=200)

In [ ]:
# R = 1
# admm_err_opt_star, admm_err_opt_reldiff, admm_const_vio, admm_f_reldiff = co.dadmm("dadmm", problem_spec, problem_data, 
#                                                                                 network_data, x_opt_star, f_star, prox_operators=prox_operators,
#                                                                                 fi_operators=fi_operators,
#                                                                                 params={"R":R}, printing=True, freq=200)

In [ ]:
# params={"R":18, "Inductance":10, "h":1.1054814693822324}
# ciradmm_err_opt_star, ciradmm_err_opt_reldiff, ciradmm_const_vio, ciradmm_f_reldiff = co.dadmm("cir_dadmm", problem_spec, problem_data, 
#                                                                                 network_data, x_opt_star, f_star, prox_operators=prox_operators,
#                                                                                 fi_operators=fi_operators,
#                                                                                 params=params, printing=True, freq=200)

In [ ]:
# apply P-EXTRA 
problem_data['alpha'] = 0.5
pg_op_norm, pg_err_opt_star, pg_err_opt_reldiff, pg_const_vio, pg_f_reldiff = p_extra_quad_no_constraint( problem_spec, problem_data, network_data, x_opt_star, f_star, printing=True, freq=200)

# p_extra_quad_no_constraint(problem_spec, problem_data, network_data, x_opt_star, f_star, printing=False, freq=200, params=None)

In [ ]:
# apply P-EXTRA 
problem_data['alpha'] = 1
pg_op_norm, pg_err_opt_star, pg_err_opt_reldiff, pg_const_vio, pg_f_reldiff = p_extra_quad_no_constraint(problem_spec, problem_data, network_data, x_opt_star, f_star, printing=True, freq=200)

In [ ]:
fname = "n6_circuit_dadmm_c"



plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams["legend.fontsize"] = 10
plt.rcParams["lines.linewidth"] = 2
plt.rcParams["lines.markersize"] = 4
plt.rcParams["legend.framealpha"] = 0.0
plt.rcParams["xtick.labelsize"] = 10
plt.rcParams["ytick.labelsize"] = 10
plt.rcParams["mathtext.fontset"] = 'cm' # default = 'dejavusans', other options = 'cm', 'stixsans', 'dejavuserif'
black, red, blue, green = 'dimgrey', 'coral', 'deepskyblue', 'gold'

# operator norm plot (log-log plot)
plt.figure(figsize=(5,4))
plt.minorticks_off()
plt.xscale("log")
plt.yscale("log")
# plt.plot(pg_f_reldiff, label='P-EXTRA', color=black, linewidth=3)
plt.plot(admm_f_reldiff, label='DADMM', color=blue, linewidth=3)
# plt.plot(ciradmm_f_reldiff, label='Circuit DADMM',  color=green, linewidth=2)
plt.plot(dadmm_c_f_reldiff, label='Circuit DADMM + C',  color=red, linewidth=2)
plt.xlabel(r"$k$")
plt.ylabel(r"$|f(x^k) - f^\star|/f^\star$")
# plt.ylabel(r"$\|\tilde{\mathbf{A}}x_k\|^2_M$")
plt.ylim(1e-10, 1e0)
plt.legend()
plt.savefig(f'plots/freldif_{fname}.pdf', dpi=300)